In [61]:
import numpy as np
import pandas as pd

In [62]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


Train Test Split

In [63]:
from sklearn.model_selection  import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']], df['bought_insurance'], test_size=0.2,random_state=42)

In [64]:
X_train_scaled = X_train.copy()
X_train_scaled['age']= X_train_scaled['age']/100

In [65]:
X_test_scaled = X_test.copy()
X_test_scaled['age']= X_test_scaled['age']/100

TesorFlow Model Building

In [66]:
from tensorflow import keras

In [67]:
model = keras.Sequential([
    keras.layers.Dense(1,input_shape= (2,), activation= 'sigmoid', kernel_initializer = 'ones', bias_initializer = 'zeros')
    ])
model.compile(optimizer='adam',loss="binary_crossentropy", metrics=['accuracy'])

In [68]:
model.fit(X_train_scaled, y_train, epochs=5000)

Epoch 1/5000
1/1 [==============================] - 1s 569ms/step - loss: 0.7428 - accuracy: 0.5000
Epoch 2/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.7424 - accuracy: 0.5000
Epoch 3/5000
1/1 [==============================] - 0s 11ms/step - loss: 0.7420 - accuracy: 0.5000
Epoch 4/5000
1/1 [==============================] - 0s 14ms/step - loss: 0.7416 - accuracy: 0.5000
Epoch 5/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.7411 - accuracy: 0.5000
Epoch 6/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.7407 - accuracy: 0.5000
Epoch 7/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.7403 - accuracy: 0.5000
Epoch 8/5000
1/1 [==============================] - 0s 17ms/step - loss: 0.7399 - accuracy: 0.5000
Epoch 9/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.7395 - accuracy: 0.5000
Epoch 10/5000
1/1 [==============================] - 0s 0s/step - loss: 0.7390 - accuracy: 0.5000
Epoch 11/5000

In [69]:
model.evaluate(X_test_scaled,y_test)

1/1 [==============================] - 0s 154ms/step - loss: 0.2647 - accuracy: 1.0000


[0.26470306515693665, 1.0]

In [70]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 76ms/step


array([[0.81626713],
       [0.75767535],
       [0.8236742 ],
       [0.18819511],
       [0.38559142],
       [0.1959822 ]], dtype=float32)

In [71]:
y_test

9     1
25    1
8     1
21    0
0     0
12    0
Name: bought_insurance, dtype: int64

In [72]:
coefficient, intercept = model.get_weights()
coefficient, intercept    #values of weigths  and bias evaluated by model

(array([[5.0183086],
        [1.1966312]], dtype=float32),
 array([-2.766541], dtype=float32))

In [73]:
# define a sigmoid function
import math
def sigmoid(x):
    return 1/(1+math.exp(-x))

In [74]:
sigmoid(0.7889)

0.6875950904656596

In [81]:
#define own prediction function with the  help of wights and bias derived by keras and tensorflow model
def prediction_fun(age, affordibility):
    sum = coefficient[0]*age + coefficient[1]*affordibility + intercept
    return sigmoid(sum)

In [82]:
X_test_scaled,  model.predict(X_test_scaled)

1/1 [==============================] - 0s 30ms/step


(     age  affordibility
 9   0.61              1
 25  0.54              1
 8   0.62              1
 21  0.26              0
 0   0.22              1
 12  0.27              0,
 array([[0.81626713],
        [0.75767535],
        [0.8236742 ],
        [0.18819511],
        [0.38559142],
        [0.1959822 ]], dtype=float32))

In [85]:
prediction_fun(.61,1)

0.816267109586199

Implementation of Gradient descent from scratch

In [91]:
# In below function the arguement 'X' is a numpy array
def sigmoid(X):
    return 1/(1+np.exp(-X))
sigmoid(np.array([3,5,6]))

array([0.95257413, 0.99330715, 0.99752738])

In [92]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    '''Below 2 lines convert 0 to 0.0000000001 and 1 to 0.99999999999
    coz log(0) and log(1) are edge cases'''
    y_predicted_new = [max(epsilon,i) for i in y_predicted]
    y_predicted_new = [min(i, 1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true * np.log(y_predicted_new) + (1-y_true)*np.log(1-y_predicted_new))


In [105]:
#Main function
def gradient_descent(age, affordibility, y_true, epochs, loss_threshold):
    w1= w2 = 1
    bias =0
    learning_rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1*age + w2*affordibility + bias
        y_predicted = sigmoid(weighted_sum)
        loss = log_loss(y_true , y_predicted)
        w1d =(1/n)* np.dot(np.transpose(age), (y_predicted- y_true))
        w2d = (1/n)* np.dot(np.transpose(affordibility), (y_predicted- y_true))
        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - learning_rate*w1d
        w2 = w2 - learning_rate*w2d
        bias =  bias - learning_rate*bias_d
        print(f'Epoch:{i}, w1:{w1}, w2:{w2} , bias:{bias} , loss:{loss}')
        if loss<= loss_threshold:
            break
    

   
    return w1, w2, bias

In [106]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'],y_train, 1000, 0.4923)

Epoch:0, w1:0.9736899318847281, w2:0.931388810977659 , bias:-0.11748951666770448 , loss:0.7428288579142563
Epoch:1, w1:0.9536535852311094, w2:0.8740290167758512 , bias:-0.21881533456146035 , loss:0.7072146449948487
Epoch:2, w1:0.9393731039296969, w2:0.8271852202997496 , bias:-0.3053620401943441 , loss:0.6814881914786812
Epoch:3, w1:0.9301932588998061, w2:0.7897792032048467 , bias:-0.37884372361582785 , loss:0.6633428084673968
Epoch:4, w1:0.9254091137248938, w2:0.7605726653866934 , bias:-0.44108236820018304 , loss:0.650742850709519
Epoch:5, w1:0.9243325693598607, w2:0.738313053647322 , bias:-0.49384257986251556 , loss:0.6420508089402462
Epoch:6, w1:0.926333296357235, w2:0.7218280753843739 , bias:-0.5387319906498417 , loss:0.6360356979531208
Epoch:7, w1:0.930858097563688, w2:0.7100747303660235 , bias:-0.5771558825717441 , loss:0.631816485354411
Epoch:8, w1:0.9374354910317362, w2:0.7021560855322683 , bias:-0.6103083840841516 , loss:0.6287844495353145
Epoch:9, w1:0.9456716791005845, w2:0.6

(5.0147687108051695, 1.227094212784905, -2.7982358620493586)